Free Parking Model predicts the availability of free parking at a person’s workplace. It is applied for people who work in zones that have parking charges

- only workers that have a usual work location outside their home and that is in a zone that has paid parking
    - PRKCST > 0
    - workers with usual work location

In [1]:
import os
import toml
import pandas as pd
import validation_data_input
import numpy as np
import plotly.express as px

# to show plotly figures in quarto HTML file
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook_connected"

config = toml.load(os.path.join(os.getcwd(), 'validation_configuration.toml'))

In [2]:
# read data

# get shared data
%store -r validation_data

config = toml.load(os.path.join(os.getcwd(), 'validation_configuration.toml'))

#per_data = validation_data_input.get_persons_data()
per_data = validation_data.persons_data_uncloned.copy()
land_use = validation_data.land_use[['zone_id', 'TAZ', 'county_id', 'PRKCST', 'OPRKCST']].copy()
# delete shared data
del validation_data

per_data = per_data.merge(land_use,how="left",left_on='workplace_zone_id',right_on='zone_id')

free_parking = per_data.loc[(per_data['workplace_zone_id']>0) & (per_data['home_zone_id']!=per_data['workplace_zone_id'])]

- number of people who work in zones that have parking charges

In [3]:
free_parking.groupby(['source'])['person_weight'].sum().reset_index()

,source,person_weight
0,model results,1.589179e+06
1,survey data,1.782115e+06
2,unweighted survey,2.897000e+03


## free parking availability for workers

In [4]:
df_plot = free_parking.groupby(['source','free_parking_at_work'])['person_weight'].sum().reset_index()

df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['person_weight']. \
    apply(lambda x: x / float(x.sum()))
df_plot['source'] = df_plot['source'].astype(pd.CategoricalDtype(['model results', 'survey data', 'unweighted survey']))

fig = px.bar(df_plot.sort_values(by=['source']), x="free_parking_at_work", y="percentage", color="source",
             barmode="group",template="simple_white",
             title="Free parking")
fig.update_layout(height=400, width=700, font=dict(size=11), yaxis_tickformat = '.0%')
fig.show()

## free parking availability for workers in zones with parking costs

In [5]:
free_parking = per_data.loc[(per_data['PRKCST']>0) & (per_data['workplace_zone_id']>0) & (per_data['home_zone_id']!=per_data['workplace_zone_id'])]

df_plot = free_parking.groupby(['source','free_parking_at_work'])['person_weight'].sum().reset_index()

df_plot['percentage'] = df_plot.groupby(['source'], group_keys=False)['person_weight']. \
    apply(lambda x: x / float(x.sum()))
df_plot['source'] = df_plot['source'].astype(pd.CategoricalDtype(['model results', 'survey data', 'unweighted survey']))

fig = px.bar(df_plot.sort_values(by=['source']), x="free_parking_at_work", y="percentage", color="source",
             barmode="group",template="simple_white",
             title="Free parking")
fig.update_layout(height=400, width=700, font=dict(size=11), yaxis_tickformat = '.0%')
fig.show()

In [6]:
print("end")

end
